In [1]:
from keras.models import Model, Input, save_model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from keras import losses
import h5py

DATA_FILE = 'data.h5'

Using TensorFlow backend.


In [2]:
with h5py.File(DATA_FILE, 'r') as f:
    
    images_training = list(f['images_training'])
    labels_training = list(f['label_training'])
        
    images_testing = list(f['images_testing'])
    labels_testing = list(f['label_testing'])

In [3]:
activation = 'relu'

input_shape = (64, 64, 1)
pic_input = Input(shape=input_shape)

img_stack = Conv2D(32, (3, 3), name='convolution0', padding='same', activation=activation)(pic_input)
img_stack = MaxPooling2D(pool_size=(2,2))(img_stack)
img_stack = Conv2D(64, (3, 3), name='convolution1', padding='same', activation=activation)(img_stack)
img_stack = MaxPooling2D(pool_size=(2,2))(img_stack)
img_stack = Conv2D(128, (3, 3), name='convolution2', padding='same', activation=activation)(img_stack)
img_stack = MaxPooling2D(pool_size=(2,2))(img_stack)
img_stack = Conv2D(123, (3, 3), name='convolution3', padding='same', activation=activation)(img_stack)
img_stack = MaxPooling2D(pool_size=(2,2))(img_stack)
img_stack = Flatten()(img_stack)
img_stack = Dropout(0.25)(img_stack)

dense_stack = Dense(128, activation=activation, name='dense0')(img_stack)
dense_stack = Dropout(0.25)(dense_stack)
dense_stack = Dense(64, activation=activation, name='dense1')(dense_stack)
dense_stack = Dropout(0.25)(dense_stack)
dense_stack = Dense(32, activation=activation, name='dense2')(dense_stack)
dense_stack = Dense(1, name='output', activation='tanh')(dense_stack)

adam = Adam()
model = Model(inputs=[pic_input], outputs=dense_stack)
model.compile(optimizer=adam, loss=losses.mean_absolute_error)


In [4]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 64, 64, 1)         0         
_________________________________________________________________
convolution0 (Conv2D)        (None, 64, 64, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
convolution1 (Conv2D)        (None, 32, 32, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
convolution2 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 8, 8, 128)         0         
__________

In [5]:
from generator import DriveDataGenerator

plateau_callback = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=0.0001, verbose=1)
model_checkpoint = ModelCheckpoint('model.h5', monitor='val_loss', save_best_only=True)
tensorboard_callback = TensorBoard(log_dir='./logs')

resize_dims = (input_shape[0], input_shape[1])
train_generator = DriveDataGenerator(images_training, labels_training, resize_dims=resize_dims)
validation_generator = DriveDataGenerator(images_testing, labels_testing, resize_dims=resize_dims)

model.fit_generator(generator=train_generator, validation_data=validation_generator,
                    epochs=500, verbose=1, use_multiprocessing=True,
                    callbacks=[plateau_callback, model_checkpoint, tensorboard_callback])

Epoch 1/500
117/117 [==============================] - 21s 181ms/step - loss: 0.9967 - val_loss: 1.0328
Epoch 2/500
117/117 [==============================] - 16s 139ms/step - loss: 0.7510 - val_loss: 0.3639
Epoch 3/500
117/117 [==============================] - 6s 47ms/step - loss: 0.3412 - val_loss: 0.3009
Epoch 4/500
117/117 [==============================] - 6s 55ms/step - loss: 0.3146 - val_loss: 0.2970
Epoch 5/500
117/117 [==============================] - 6s 54ms/step - loss: 0.3021 - val_loss: 0.2789
Epoch 6/500
117/117 [==============================] - 6s 51ms/step - loss: 0.2942 - val_loss: 0.2640
Epoch 7/500
117/117 [==============================] - 5s 45ms/step - loss: 0.2895 - val_loss: 0.2678
Epoch 8/500
117/117 [==============================] - 6s 49ms/step - loss: 0.2903 - val_loss: 0.2733
Epoch 9/500
117/117 [==============================] - 6s 52ms/step - loss: 0.2862 - val_loss: 0.2624
Epoch 10/500
117/117 [==============================] - 6s 51ms/step - loss: 0

117/117 [==============================] - 6s 55ms/step - loss: 0.2259 - val_loss: 0.2152
Epoch 78/500
117/117 [==============================] - 6s 50ms/step - loss: 0.2244 - val_loss: 0.2106
Epoch 79/500
117/117 [==============================] - 6s 51ms/step - loss: 0.2207 - val_loss: 0.2142
Epoch 80/500
117/117 [==============================] - 6s 53ms/step - loss: 0.2187 - val_loss: 0.2131
Epoch 81/500
117/117 [==============================] - 6s 52ms/step - loss: 0.2175 - val_loss: 0.2266
Epoch 82/500
117/117 [==============================] - 6s 52ms/step - loss: 0.2212 - val_loss: 0.2107
Epoch 83/500
117/117 [==============================] - 6s 51ms/step - loss: 0.2171 - val_loss: 0.2193
Epoch 84/500
117/117 [==============================] - 6s 50ms/step - loss: 0.2203 - val_loss: 0.2088
Epoch 85/500
117/117 [==============================] - 5s 45ms/step - loss: 0.2135 - val_loss: 0.2196
Epoch 86/500
117/117 [==============================] - 6s 51ms/step - loss: 0.2181 - 

117/117 [==============================] - 6s 49ms/step - loss: 0.1937 - val_loss: 0.2180
Epoch 157/500
117/117 [==============================] - 6s 49ms/step - loss: 0.1959 - val_loss: 0.2005
Epoch 158/500
117/117 [==============================] - 5s 47ms/step - loss: 0.1948 - val_loss: 0.2210
Epoch 159/500
117/117 [==============================] - 5s 46ms/step - loss: 0.1964 - val_loss: 0.2040
Epoch 160/500
117/117 [==============================] - 5s 47ms/step - loss: 0.2000 - val_loss: 0.2122
Epoch 161/500
117/117 [==============================] - 6s 53ms/step - loss: 0.1982 - val_loss: 0.2143
Epoch 162/500
117/117 [==============================] - 6s 54ms/step - loss: 0.1926 - val_loss: 0.2113
Epoch 163/500
117/117 [==============================] - 6s 54ms/step - loss: 0.1950 - val_loss: 0.2104
Epoch 164/500
117/117 [==============================] - 6s 56ms/step - loss: 0.1916 - val_loss: 0.2145
Epoch 165/500
117/117 [==============================] - 6s 55ms/step - loss: 

Epoch 235/500
117/117 [==============================] - 6s 52ms/step - loss: 0.1764 - val_loss: 0.2091
Epoch 236/500
117/117 [==============================] - 5s 41ms/step - loss: 0.1772 - val_loss: 0.2094
Epoch 237/500
117/117 [==============================] - 6s 47ms/step - loss: 0.1763 - val_loss: 0.2125
Epoch 238/500
117/117 [==============================] - 5s 45ms/step - loss: 0.1746 - val_loss: 0.2051
Epoch 239/500
117/117 [==============================] - 5s 45ms/step - loss: 0.1764 - val_loss: 0.2141
Epoch 240/500
117/117 [==============================] - 5s 45ms/step - loss: 0.1758 - val_loss: 0.2096
Epoch 241/500
117/117 [==============================] - 5s 45ms/step - loss: 0.1786 - val_loss: 0.2072
Epoch 242/500
117/117 [==============================] - 7s 57ms/step - loss: 0.1786 - val_loss: 0.2145
Epoch 243/500
117/117 [==============================] - 6s 50ms/step - loss: 0.1758 - val_loss: 0.2100
Epoch 244/500
117/117 [==============================] - 6s 50ms

117/117 [==============================] - 7s 56ms/step - loss: 0.1634 - val_loss: 0.2051
Epoch 314/500
117/117 [==============================] - 7s 61ms/step - loss: 0.1634 - val_loss: 0.2083
Epoch 315/500
117/117 [==============================] - 6s 51ms/step - loss: 0.1607 - val_loss: 0.2043
Epoch 316/500
117/117 [==============================] - 6s 51ms/step - loss: 0.1608 - val_loss: 0.2028
Epoch 317/500
117/117 [==============================] - 6s 50ms/step - loss: 0.1649 - val_loss: 0.2144
Epoch 318/500
117/117 [==============================] - 6s 49ms/step - loss: 0.1637 - val_loss: 0.2136
Epoch 319/500
117/117 [==============================] - 6s 49ms/step - loss: 0.1617 - val_loss: 0.2079
Epoch 320/500
117/117 [==============================] - 6s 50ms/step - loss: 0.1611 - val_loss: 0.2044
Epoch 321/500
117/117 [==============================] - 6s 52ms/step - loss: 0.1624 - val_loss: 0.2100
Epoch 322/500
117/117 [==============================] - 6s 51ms/step - loss: 

Epoch 392/500
117/117 [==============================] - 6s 55ms/step - loss: 0.1514 - val_loss: 0.2106
Epoch 393/500
117/117 [==============================] - 6s 54ms/step - loss: 0.1556 - val_loss: 0.2048
Epoch 394/500
117/117 [==============================] - 6s 55ms/step - loss: 0.1497 - val_loss: 0.2032
Epoch 395/500
117/117 [==============================] - 6s 54ms/step - loss: 0.1510 - val_loss: 0.2019
Epoch 396/500
117/117 [==============================] - 7s 56ms/step - loss: 0.1490 - val_loss: 0.2072
Epoch 397/500
117/117 [==============================] - 6s 54ms/step - loss: 0.1508 - val_loss: 0.2110
Epoch 398/500
117/117 [==============================] - 6s 54ms/step - loss: 0.1484 - val_loss: 0.2014
Epoch 399/500
117/117 [==============================] - 6s 54ms/step - loss: 0.1523 - val_loss: 0.2090
Epoch 400/500
117/117 [==============================] - 6s 54ms/step - loss: 0.1490 - val_loss: 0.2002
Epoch 401/500
117/117 [==============================] - 6s 55ms

117/117 [==============================] - 7s 56ms/step - loss: 0.1405 - val_loss: 0.1989
Epoch 471/500
117/117 [==============================] - 7s 60ms/step - loss: 0.1420 - val_loss: 0.2057
Epoch 472/500
117/117 [==============================] - 6s 55ms/step - loss: 0.1465 - val_loss: 0.2065
Epoch 473/500
117/117 [==============================] - 6s 55ms/step - loss: 0.1416 - val_loss: 0.2107
Epoch 474/500
117/117 [==============================] - 6s 54ms/step - loss: 0.1433 - val_loss: 0.2087
Epoch 475/500
117/117 [==============================] - 6s 54ms/step - loss: 0.1440 - val_loss: 0.1987
Epoch 476/500
117/117 [==============================] - 6s 54ms/step - loss: 0.1384 - val_loss: 0.2110
Epoch 477/500
117/117 [==============================] - 6s 55ms/step - loss: 0.1433 - val_loss: 0.2125
Epoch 478/500
117/117 [==============================] - 6s 55ms/step - loss: 0.1414 - val_loss: 0.2057
Epoch 479/500
117/117 [==============================] - 6s 54ms/step - loss: 